In [1]:
!pip install rasterio rioxarray geopandas xarray shapely scikit-learn opencv-python imageio folium pyproj

import numpy as np
import matplotlib.pyplot as plt
import rasterio
import rioxarray as rxr
import geopandas as gpd
import xarray as xr
from pyproj import CRS
from scipy.ndimage import shift, binary_dilation
import imageio

You should consider upgrading via the 'C:\Users\Cindy\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
TARGET_CRS = "EPSG:4326"
TARGET_RES = (0.01, 0.01)  # adjust resolution based on file size

In [ ]:
def load_raster(path):
    da = rxr.open_rasterio(path, masked=True).squeeze()
    # Standardize CRS & resolution
    da = da.rio.reproject(TARGET_CRS, resolution=TARGET_RES)
    return da

# TODO: Replace these with actual inputs (JPGs, GeoTIFFs, netCDF)
# These are dummy placeholders:
day0 = xr.DataArray(np.random.rand(300, 300), dims=("y", "x"))
day1 = xr.DataArray(np.random.rand(300, 300), dims=("y", "x"))
day2 = xr.DataArray(np.random.rand(300, 300), dims=("y", "x"))
day3 = xr.DataArray(np.random.rand(300, 300), dims=("y", "x"))

ice_hist = xr.concat([day0, day1, day2], dim="time")

In [ ]:
# TODO: Replace with real shipping lane shapefile
shipping = gpd.GeoDataFrame(geometry=[])

In [ ]:
# TODO: replace with real data ingestion
temp_forecast = [-6, -2, 1]    # °C per day
wind_speed_fc = [8, 10, 6]     # m/s
wind_dir_fc   = [270, 300, 280]  # degrees FROM, oceanographic

In [ ]:
def temp_delta(temp_c):
    if temp_c < -5: return 0.05
    if temp_c > 0: return -0.10
    return 0.0

def forecast_step(conc, temp, wind_speed, wind_dir, alpha=0.02):
    """Apply temp melt/growth + drift shift."""
    delta = temp_delta(temp)
    
    # thermal effect
    conc = conc + delta
    
    # morphological growth/melt
    if delta > 0:   # growth
        conc = np.maximum(conc, binary_dilation(conc > 0.1).astype(float) * 0.2)
    else:           # melt
        conc *= 0.9

    # drift based on wind
    dx = alpha * wind_speed * np.cos(np.deg2rad(wind_dir))
    dy = alpha * wind_speed * np.sin(np.deg2rad(wind_dir))
    conc = shift(conc, shift=(dy, dx), order=1, mode='nearest')

    return np.clip(conc, 0, 1)

In [ ]:
forecast = []
conc = ice_hist[-1].data.copy()

forecast.append(conc.copy())  # Day 0 = initial

for i in range(3):
    conc = forecast_step(conc, temp_forecast[i], wind_speed_fc[i], wind_dir_fc[i])
    forecast.append(conc.copy())

forecast = np.stack(forecast, axis=0)

In [ ]:
def plot_forecast(conc, day_num):
    plt.figure(figsize=(6,5))
    plt.imshow(conc, origin='lower', vmin=0, vmax=1, cmap='Blues')
    plt.colorbar(label="Ice Concentration")
    
    # overlay shipping lanes
    try:
        if len(shipping) > 0:
            for geom in shipping.geometry:
                xs, ys = geom.xy
                plt.plot(xs, ys, color='red', linewidth=1)
    except:
        pass
    
    plt.title(f"Forecast Day {day_num}")
    plt.axis("off")
    plt.savefig(f"forecast_day_{day_num}.jpg", dpi=200)
    plt.show()

In [ ]:
for d in range(4):
    plot_forecast(forecast[d], d)

In [ ]:
images = []
for d in range(4):
    img = imageio.imread(f"forecast_day_{d}.jpg")
    images.append(img)

imageio.mimsave("forecast_animation.gif", images, fps=1)

print("✓ Finished: JPGs + GIF generated")